<a id='1'></a>
# 1. Import packages

In [12]:
from keras.models import Sequential, Model
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.applications import *
import keras.backend as K
from tensorflow.contrib.distributions import Beta
import tensorflow as tf
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
from image_augmentation import random_transform
from image_augmentation import random_warp
from utils import get_image_paths, load_images, stack_images
from pixel_shuffler import PixelShuffler
from instance_normalization import InstanceNormalization

In [0]:
import time
import os
import numpy as np
from PIL import Image
import cv2
import glob
from random import randint, shuffle
from IPython.display import clear_output
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib notebook

In [0]:
from umeyama import umeyama

In [16]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16097123334067914334, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11297803469
 locality {
   bus_id: 1
 }
 incarnation: 18276316126844872470
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

<a id='3'></a>
# 3. Import VGGFace
(Skip this part if you don't want to apply perceptual loss)

If you got error ```_obtain_input_shape(...)``` error, this is because your keras version is older than vggface requirement. 

Modify ```_obtain_input_shape(...)``` in ```keras_vggface/models.py``` will solve the problem. The following is what worked for me:

```python
input_shape = _obtain_input_shape(input_shape,
                                  default_size=224,
                                  min_size=197,
                                  data_format=K.image_data_format(),
                                  include_top=include_top)
```

In [0]:
from keras_vggface.vggface import VGGFace

In [0]:
vggface = VGGFace(include_top=False, model='resnet50', input_shape=(224, 224, 3))

<a id='4'></a>
# 4. Config

mixup paper: https://arxiv.org/abs/1710.09412

Default training data directories: `./faceA/` and `./faceB/`

In [0]:
K.set_learning_phase(1)

In [0]:
channel_axis=-1
channel_first = False

In [0]:
IMAGE_SHAPE = (64, 64, 3)
nc_in = 3 # number of input channels of generators
nc_D_inp = 6 # number of input channels of discriminators

use_perceptual_loss = True
use_lsgan = True
use_instancenorm = False
use_mixup = True
mixup_alpha = 0.2 # 0.2

batchSize = 8
lrD = 2e-4 # Discriminator learning rate
lrG = 2e-4 # Generator learning rate

# Path of training images
img_dirA = './faceA/*.*'
img_dirB = './faceB/*.*'

<a id='5'></a>
# 5. Define models

In [0]:
def __conv_init(a):
    print("conv_init", a)
    k = RandomNormal(0, 0.02)(a) # for convolution kernel
    k.conv_weight = True    
    return k
conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02) # for batch normalization

In [0]:
def conv_block(input_tensor, f, k=3, strides=2, dilation_rate=1, use_inst_norm=True):
    x = input_tensor
    x = Conv2D(f, kernel_size=k, strides=strides, dilation_rate=dilation_rate, kernel_initializer=conv_init, use_bias=True, padding="same")(x)
    if use_inst_norm:
        x = InstanceNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    return x

def conv_block_d(input_tensor, f, use_inst_norm=True):
    x = input_tensor
    x = Conv2D(f, kernel_size=4, strides=2, kernel_initializer=conv_init, use_bias=True, padding="same")(x)
    if use_inst_norm:
        x = InstanceNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def res_block(input_tensor, f):
    x = input_tensor
    x = Conv2D(f, kernel_size=5, kernel_initializer=conv_init, use_bias=True, padding="same")(x)
    x = InstanceNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(f, kernel_size=5, kernel_initializer=conv_init, use_bias=True, padding="same")(x)
    x = InstanceNormalization()(x)
    x = add([x, input_tensor])
    return x

def upscale_ps(filters, dilation_rate=1, use_norm=True):
    def block(x):
        x = Conv2D(filters*4, kernel_size=3, dilation_rate=dilation_rate, use_bias=True, kernel_initializer=RandomNormal(0, 0.02), padding='same')(x)
        x = InstanceNormalization()(x)
        x = LeakyReLU(0.1)(x)
        x = PixelShuffler()(x)
        return x
    return block

def Discriminator(nc_in, input_size=64):
    inp = Input(shape=(input_size, input_size, nc_in))
    #x = GaussianNoise(0.05)(inp)
    x = conv_block_d(inp, 64, False)
    x = conv_block_d(x, 128)
    x = conv_block_d(x, 256)
    out = Conv2D(1, kernel_size=4, kernel_initializer=conv_init, use_bias=False, padding="same", activation="sigmoid")(x)   
    return Model(inputs=[inp], outputs=out)

def Discriminator2(nc_in, input_size=64):
    inp = Input(shape=(input_size, input_size, nc_in))
    #x = GaussianNoise(0.05)(inp)
    x = conv_block_d(inp, 64, False)
    x = conv_block_d(x, 128)
    x = conv_block_d(x, 256)
    x = conv_block_d(x, 512)
    out = Conv2D(1, kernel_size=3, kernel_initializer=conv_init, use_bias=False, padding="same", activation="sigmoid")(x)   
    return Model(inputs=[inp], outputs=out) 
  
def Discriminator3(nc_in, input_size=64):
    inp = Input(shape=(input_size, input_size, nc_in))
    x = conv_block_d(inp, 32, False)
    x = mixnet_block(x, 12)
    x = Conv2D(64, kernel_size=1, kernel_initializer=conv_init, use_bias=False, padding="same")(x)
    x = AveragePooling2D()(x)
    x = mixnet_block(x, 24)
    x = Conv2D(128, kernel_size=1, kernel_initializer=conv_init, use_bias=False, padding="same")(x)
    x = AveragePooling2D()(x)
    x = mixnet_block(x, 24)
    x = Conv2D(256, kernel_size=1, kernel_initializer=conv_init, use_bias=False, padding="same")(x)
    x = AveragePooling2D()(x)
    out = Conv2D(1, kernel_size=3, kernel_initializer=conv_init, use_bias=False, padding="same", activation="sigmoid")(x)   
    return Model(inputs=[inp], outputs=out) 

def Discriminator_code():
    inp = Input(shape=(1024, ))
    x = Dense(256)(inp)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    x = Dense(128)(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    out = Dense(1, activation='sigmoid')(x)   
    return Model(inputs=[inp], outputs=out)

def Encoder(nc_in=3, input_size=64):
    def l2_norm(x):
        epsilon = 1e-12
        x_norm = K.sqrt(K.sum(K.square(x)))
        return x / (x_norm + epsilon)  
  
    inp = Input(shape=(input_size, input_size, nc_in))
    x = Conv2D(64, kernel_size=5, kernel_initializer=conv_init, use_bias=False, padding="same")(inp)
    x = LeakyReLU(0.1)(x)
    x = conv_block(x,128)
    x = conv_block(x,256)
    x = conv_block(x,512)
    x = conv_block(x,1024)
    x = Dense(1024)(Flatten()(x))
    x = InstanceNormalization()(x)
    x = LeakyReLU(0.1)(x)
    x = Dense(1024)(x)
    x = Lambda(l2_norm)(x)
    code = x
    x = Dense(4*4*1024)(x)
    x = Reshape((4, 4, 1024))(x)
    out = upscale_ps(512, dilation_rate=1)(x)
    return Model(inputs=inp, outputs=[out, code])

def Decoder_ps(nc_in=512, input_size=8):
    inp = Input(shape=(input_size, input_size, nc_in))
    code = Input(shape=(1024,))
    x = inp
    x = upscale_ps(256)(x)
    x = upscale_ps(128)(x)
    x = upscale_ps(64)(x)
    x = res_block(x, 64)
    x, _ = mixed_scaled_dense_network(x, 16, 64, 4)
    alpha = Lambda(lambda x: x[:,:,:,0:1])(x)
    alpha = Activation("sigmoid")(alpha)
    bgr = Lambda(lambda x: x[:,:,:,1:])(x)
    bgr = Activation("tanh")(bgr)
    out = concatenate([alpha, bgr])
    return Model([inp, code], [out, code])    
  
def mixed_scaled_dense_network(input_tensor, num_layers=32, nc_in=3, nc_out=1):
    """
    Inefficient implementation of paper: A mixed-scale dense convolutional neural network for image analysis
    http://www.pnas.org/content/115/2/254
    """
    msd_layers = {}
    x = input_tensor
    msd_layers["input"] = x
    for i in range(num_layers):
        dilation = (i % 10) + 1
        msd_layers["layer{0}".format(i)] = Conv2D(1, kernel_size=3, strides=1, dilation_rate=dilation, 
                                                  kernel_initializer=conv_init, padding="same")(x)
        for j in range(i):
            dilation = ((i + j) %10) + 1
            conv_3x3 = Conv2D(1, kernel_size=3, strides=1, dilation_rate=dilation,
                              kernel_initializer=conv_init, use_bias=True, padding="same")(msd_layers["layer{0}".format(j)])
            msd_layers["layer{0}".format(i)] = add([msd_layers["layer{0}".format(i)], conv_3x3])
        msd_layers["layer{0}".format(i)] = Activation("relu")(msd_layers["layer{0}".format(i)])
    
    concat_all = x
    for i in range(num_layers):
        concat_all = concatenate([concat_all, msd_layers["layer{0}".format(i)]])
    msd_layers["merge_concat_all"] = concat_all
    out = Conv2D(nc_out, kernel_size=1, kernel_initializer=conv_init, padding="same")(concat_all)
    msd_layers["output"] = out
    
    return out, msd_layers

def mixnet_block(input_tensor, nc_concat=12, type="fixed"):
    """
    Inspired by: Mixed Link Networks (https://arxiv.org/abs/1802.01808) 
    Inner-loop has different implementation.
    """
    def se_block(input_tensor, compress_rate = 16):
        num_channels = int(input_tensor.shape[-1]) # Tensorflow backend
        bottle_neck = int(num_channels//compress_rate)

        se_branch = GlobalAveragePooling2D()(input_tensor)
        se_branch = Dense(bottle_neck, activation='relu', kernel_initializer='he_normal', use_bias=False)(se_branch)
        se_branch = Dense(num_channels, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se_branch)
        x = input_tensor 
        out = multiply([x, se_branch])
        return out
      
    x = input_tensor
    nc_in = int(input_tensor.shape[-1])
    
    k2 = nc_concat
    x_out = Conv2D(nc_in//2, kernel_size=1, kernel_initializer=conv_init, use_bias=True, padding="same")(x)
    x_out = InstanceNormalization()(x_out)
    x_out = LeakyReLU(alpha=0.1)(x_out)
    x_out = Conv2D(k2, kernel_size=3, kernel_initializer=conv_init, use_bias=True, padding="same")(x_out)
    x_out = InstanceNormalization()(x_out)
    x_out = LeakyReLU(alpha=0.1)(x_out)    
    
    if type == "fixed":
        k1 = nc_in // 2
        x_in = Conv2D(nc_in//4, kernel_size=1, kernel_initializer=conv_init, use_bias=True, padding="same")(x)
        x_in = InstanceNormalization()(x_in)
        x_in = LeakyReLU(alpha=0.1)(x_in)
        x_in = Conv2D(k1, kernel_size=3, kernel_initializer=conv_init, use_bias=True, padding="same")(x_in)
        #x_in = Conv2D(nc_in, kernel_size=3, kernel_initializer=conv_init, use_bias=True, padding="same")(x_in)
        x_in = InstanceNormalization()(x_in)
        x_in = se_block(x_in)
        x_in_add = Lambda(lambda x: x[0][:,:,:, -k1:] + x[1])([x, x_in])
        x_in_id = Lambda(lambda x: x[:,:,:, :k1])(x)
        x_in = concatenate([x_in_id, x_in_add])
        #x_in = add([x, x_in])
        x_in = LeakyReLU(alpha=0.1)(x_in)
    elif type == "unfixed":
        k1 = nc_concat
        x_in = Conv2D(nc_in//2, kernel_size=1, kernel_initializer=conv_init, use_bias=True, padding="same")(x)
        x_in = InstanceNormalization()(x_in)
        x_in = LeakyReLU(alpha=0.1)(x_in)
        x_in = Conv2D(k1, kernel_size=3, kernel_initializer=conv_init, use_bias=True, padding="same")(x_in)
        x_in = InstanceNormalization()(x_in)
        x_in_add = Lambda(lambda x: x[0][:,:,:, -k1:] + x[1])([x, x_in])
        x_in_id = Lambda(lambda x: x[:,:,:, :k1])(x)
        x_in = concatenate([x_in_id, x_in_add])
        x_in = LeakyReLU(alpha=0.1)(x_in)

    out = concatenate([x_in, x_out])
    return out   
  

In [0]:
encoder = Encoder()
decoder_A = Decoder_ps()
decoder_B = Decoder_ps()

x = Input(shape=IMAGE_SHAPE)

netGA = Model(x, decoder_A(encoder(x)))
netGB = Model(x, decoder_B(encoder(x)))

In [0]:
netDA = Discriminator2(nc_D_inp)
netDB = Discriminator2(nc_D_inp)
netDA2 = Discriminator2(nc_D_inp//2)
netDB2 = Discriminator2(nc_D_inp//2)

In [0]:
netD_code = Discriminator_code()

<a id='6'></a>
# 6. Load Models

In [14]:
try:
    encoder.load_weights("models/encoder.h5")
    decoder_A.load_weights("models/decoder_A.h5")
    decoder_B.load_weights("models/decoder_B.h5")
    netDA.load_weights("models/netDA.h5") 
    netDB.load_weights("models/netDB.h5") 
    netDA2.load_weights("models/netDA2.h5") 
    netDB2.load_weights("models/netDB2.h5") 
    netD_code.load_weights("models/netD_code.h5")
    print ("model loaded.")
except:
    print ("Failed loading. Something went wrong when loading weights files.")
    pass

model loaded.


<a id='7'></a>
# 7. Define Inputs/Outputs Variables


In [0]:
def cycle_variables(netG):
    distorted_input = netG.inputs[0]
    fake_output = netG.outputs[0]
    alpha = Lambda(lambda x: x[:,:,:, :1])(fake_output)
    rgb = Lambda(lambda x: x[:,:,:, 1:])(fake_output)
    
    masked_fake_output = alpha * rgb + (1-alpha) * distorted_input 

    fn_generate = K.function([distorted_input], [masked_fake_output])
    fn_mask = K.function([distorted_input], [concatenate([alpha, alpha, alpha])])
    fn_abgr = K.function([distorted_input], [concatenate([alpha, rgb])])
    fn_bgr = K.function([distorted_input], [rgb])
    return distorted_input, fake_output, alpha, fn_generate, fn_mask, fn_abgr, fn_bgr

In [0]:
distorted_A, fake_A, mask_A, path_A, path_mask_A, path_abgr_A, path_bgr_A = cycle_variables(netGA)
distorted_B, fake_B, mask_B, path_B, path_mask_B, path_abgr_B, path_bgr_B = cycle_variables(netGB)
real_A = Input(shape=IMAGE_SHAPE)
real_B = Input(shape=IMAGE_SHAPE)

<a id='8'></a>
# 8. Define Loss Function

In [0]:
if use_lsgan:
    loss_fn = lambda output, target : K.mean(K.abs(K.square(output-target)))
else:
    loss_fn = lambda output, target : -K.mean(K.log(output+1e-12)*target+K.log(1-output+1e-12)*(1-target))

loss_fn_bce = lambda output, target : -K.mean(K.log(output+1e-12)*target+K.log(1-output+1e-12)*(1-target))

In [0]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html
def cos_distance(x1, x2):
    x1 = K.l2_normalize(x1, axis=-1)
    x2 = K.l2_normalize(x2, axis=-1)
    return K.mean(1 - K.sum((x1 * x2), axis=-1))

In [0]:
def first_order(x, axis=1):
    img_nrows = x.shape[1]
    img_ncols = x.shape[2]
    if axis == 1:
        return K.abs(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    elif axis == 2:
        return K.abs(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    else:
        return None   

In [22]:
# ========== Define Perceptual Loss Model==========
if use_perceptual_loss:
    print ("Using perceptual loss.")
    vggface.trainable = False
    out_size55 = vggface.layers[36].output
    out_size28 = vggface.layers[78].output
    out_size7 = vggface.layers[-2].output
    vggface_feat = Model(vggface.input, [out_size55, out_size28, out_size7])
    vggface_feat.trainable = False
    netDA_feat = netDB_feat = 0
else:
    print ("Not using perceptual loss.")
    netDA_feat = netDB_feat = vggface_feat = None

Not using perceptual loss.


In [0]:
def define_loss(netD, netD2, netD_feat, netD_code, netG, real, fake_argb, distorted, vggface_feat=None, domain="A"):   
    alpha = Lambda(lambda x: x[:,:,:, :1])(fake_argb)
    fake_rgb = Lambda(lambda x: x[:,:,:, 1:])(fake_argb)
    fake = alpha * fake_rgb + (1-alpha) * distorted
    
    if use_mixup:
        # Loss of masked output
        dist = Beta(mixup_alpha, mixup_alpha)
        lam = dist.sample()
        # ==========
        mixup = lam * concatenate([real, distorted]) + (1 - lam) * concatenate([fake, distorted])
        # ==========
        output_mixup = netD(mixup)
        loss_D = loss_fn(output_mixup, lam * K.ones_like(output_mixup)) 
        loss_G = .5 * loss_fn(output_mixup, (1 - lam) * K.ones_like(output_mixup))
        
        # Loss of raw output
        #real_shuffled = Lambda(lambda x: tf.random_shuffle(x))(real)
        lam2 = dist.sample()
        mixup2 = lam2 * real + (1 - lam2) * fake_rgb
        output2_mixup = netD2(mixup2)  
        loss_D2 = loss_fn(output2_mixup, lam2 * K.ones_like(output2_mixup))
        loss_G += .5 * loss_fn(output2_mixup, (1 - lam) * K.ones_like(output2_mixup))          
        
        # Domain adversarial loss
        real_code = netG([real])[1]
        rec_code = netG([fake_rgb])[1]
        output_real_code = netD_code([real_code])
        # Target of domain A = 1, domain B = 0
        if domain == "A":
            loss_D_code = loss_fn_bce(output_real_code, K.ones_like(output_real_code))
            loss_G += .03 * loss_fn(output_real_code, K.zeros_like(output_real_code))
        elif domain == "B":
            loss_D_code = loss_fn_bce(output_real_code, K.zeros_like(output_real_code))
            loss_G += .03 * loss_fn(output_real_code, K.ones_like(output_real_code)) 
            
        # semantic consistency loss    
        loss_G += 1. * cos_distance(rec_code, real_code)
        
    else:
        print ("use_mixup=False is not supported in v2.1!")
        
    # ==========  
    # L1 loss
    loss_G += 3 * K.mean(K.abs(fake_rgb - real))    
    # ==========
    
    # Edge loss (similar with total variation loss)
    loss_G += 1 * K.mean(K.abs(first_order(fake_rgb, axis=1) - first_order(real, axis=1)))
    loss_G += 1 * K.mean(K.abs(first_order(fake_rgb, axis=2) - first_order(real, axis=2)))
    
    loss_D_feat = 0
    # Perceptual Loss
    if not vggface_feat is None:
        def preprocess_vggface(x):
            x = (x + 1)/2 * 255 # channel order: BGR
            x -= [91.4953, 103.8827, 131.0912]
            return x
        pl_params = (0.03, 0.2, 0.3)
        real_sz224 = tf.image.resize_images(real, [224, 224])
        real_sz224 = Lambda(preprocess_vggface)(real_sz224)
        # ==========
        fake_sz224 = tf.image.resize_images(fake, [224, 224]) 
        fake_sz224 = Lambda(preprocess_vggface)(fake_sz224)
        # ==========   
        real_feat55, real_feat28, real_feat7 = vggface_feat(real_sz224)
        fake_feat55, fake_feat28, fake_feat7  = vggface_feat(fake_sz224)    
        loss_G += pl_params[0] * K.mean(K.square(fake_feat7 - real_feat7))
        loss_G += pl_params[1] * K.mean(K.abs(fake_feat28 - real_feat28))
        loss_G += pl_params[2] * K.mean(K.abs(fake_feat55 - real_feat55))    
        
        # adversarial loss on feature level
        #real_feat28_resized = tf.image.resize_images(real_feat28, [55, 55])
        #fake_feat28_resized = tf.image.resize_images(fake_feat28, [55, 55])        
        #input_real_D_feat = concatenate([real_feat55, real_feat28_resized])
        #input_fake_D_feat = concatenate([fake_feat55, fake_feat28_resized])        
        #dist1 = Beta(mixup_alpha/2, mixup_alpha/2)
        #lam1 = dist1.sample()        
        #mixup_feat = lam1 * input_real_D_feat + (1 - lam1) * input_fake_D_feat
        #output_mixup_feat = netD_feat(mixup_feat)
        #loss_D_feat = loss_fn(output_mixup_feat, lam1 * K.ones_like(output_mixup_feat)) 
        #loss_G += .05 * loss_fn(output_mixup_feat, (1 - lam1) * K.ones_like(output_mixup_feat))
    return loss_D, loss_D2, loss_G, loss_D_feat, loss_D_code

In [0]:
loss_DA, loss_DA2, loss_GA, loss_DA_feat, loss_DA_code = define_loss(netDA, netDA2, netDA_feat, netD_code, netGA, 
                                                           real_A, fake_A, distorted_A, vggface_feat, domain="A")
loss_DB, loss_DB2, loss_GB, loss_DB_feat, loss_DB_code = define_loss(netDB, netDB2, netDB_feat, netD_code, netGB,
                                                           real_B, fake_B, distorted_B, vggface_feat, domain="B")

loss_GA += 1e-3 * K.mean(K.abs(mask_A))
loss_GB += 1e-3 * K.mean(K.abs(mask_B))

w_fo = 0.01
loss_GA += w_fo * K.mean(first_order(mask_A, axis=1))
loss_GA += w_fo * K.mean(first_order(mask_A, axis=2))
loss_GB += w_fo * K.mean(first_order(mask_B, axis=1))
loss_GB += w_fo * K.mean(first_order(mask_B, axis=2))

In [0]:
weightsDA = netDA.trainable_weights
weightsDA2 = netDA2.trainable_weights
weightsGA = netGA.trainable_weights
weightsDB = netDB.trainable_weights
weightsDB2 = netDB2.trainable_weights
weightsGB = netGB.trainable_weights
weightsD_code = netD_code.trainable_weights

# Adam(..).get_updates(...)

"""
# Using the following update function spped up training time (per iter.) by ~15%.
training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsDA+weightsDA2+weightsD_code,[],loss_DA+loss_DA2+loss_DA_code)
netDA_train = K.function([distorted_A, real_A],[loss_DA+loss_DA2+loss_DA_code], training_updates)
"""

training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsDA,[],loss_DA)
netDA_train = K.function([distorted_A, real_A],[loss_DA], training_updates)
training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsDA2,[],loss_DA2)
netDA2_train = K.function([distorted_A, real_A],[loss_DA2], training_updates)
training_updates = Adam(lr=lrG, beta_1=0.5).get_updates(weightsGA,[], loss_GA)
netGA_train = K.function([distorted_A, real_A], [loss_GA], training_updates)

training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsDB,[],loss_DB)
netDB_train = K.function([distorted_B, real_B],[loss_DB], training_updates)
training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsDB2,[],loss_DB2)
netDB2_train = K.function([distorted_B, real_B],[loss_DB2], training_updates)
training_updates = Adam(lr=lrG, beta_1=0.5).get_updates(weightsGB,[], loss_GB)
netGB_train = K.function([distorted_B, real_B], [loss_GB], training_updates)

training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsD_code,[], loss_DA_code)
netDA_code_train = K.function([distorted_A, real_A],[loss_DA_code], training_updates)
training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsD_code,[], loss_DB_code)
netDB_code_train = K.function([distorted_B, real_B],[loss_DB_code], training_updates)

#if use_perceptual_loss:
#    weightsDA_feat = netDA_feat.trainable_weights
#    weightsDB_feat = netDB_feat.trainable_weights
#    training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsDA_feat,[],loss_DA_feat)
#    netDA_feat_train = K.function([distorted_A, real_A],[loss_DA_feat], training_updates)
#    training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(weightsDB_feat,[],loss_DB_feat)
#    netDB_feat_train = K.function([distorted_B, real_B],[loss_DB_feat], training_updates)

<a id='9'></a>
# 9. Utils For Loading/Displaying Images

In [0]:
def load_data(file_pattern):
    return glob.glob(file_pattern)

def random_channel_shift(x, intensity=10):
    x = cv2.cvtColor(x, cv2.COLOR_BGR2HSV).astype('float32')
    x += np.array([np.random.uniform(-intensity, intensity), 
                   np.random.uniform(-3*intensity, 3*intensity), 
                   np.random.uniform(-3*intensity, 3*intensity)])
    x = np.clip(x, 0, 255).astype('uint8')
    x = cv2.cvtColor(x, cv2.COLOR_HSV2BGR)
    return x
  
def random_cutout(x, intensity=0.2):
    h, w, c = x.shape
    cutout_range = np.min([h//2, w//2])
    cutout_x0 = np.random.randint(h)
    cutout_y0 = np.random.randint(w)
    cutout_x1 = np.min([cutout_x0+np.random.randint(cutout_range)+10, h])
    cutout_y1 = np.min([cutout_y0+np.random.randint(cutout_range)+10, w])
    cutout_h = cutout_x1 - cutout_x0
    cutout_w = cutout_y1 - cutout_y0
    x[cutout_x0:cutout_x1, cutout_y0:cutout_y1, :] = np.random.normal(0., intensity, size=(cutout_h, cutout_w, c))
    return x

def random_warp_rev(image):
    assert image.shape == (256,256,3)
    rand_coverage = np.random.randint(16) + 80
    range_ = np.linspace(128-rand_coverage, 128+rand_coverage, 5)
    mapx = np.broadcast_to(range_, (5,5))
    mapy = mapx.T
    mapx = mapx + np.random.normal( size=(5,5), scale=6)
    mapy = mapy + np.random.normal( size=(5,5), scale=6)
    interp_mapx = cv2.resize( mapx, (80,80) )[8:72,8:72].astype('float32')
    interp_mapy = cv2.resize( mapy, (80,80) )[8:72,8:72].astype('float32')
    warped_image = cv2.remap(image, interp_mapx, interp_mapy, cv2.INTER_LINEAR)
    src_points = np.stack( [ mapx.ravel(), mapy.ravel() ], axis=-1)
    dst_points = np.mgrid[0:65:16,0:65:16].T.reshape(-1,2)
    mat = umeyama(src_points, dst_points, True)[0:2]
    target_image = cv2.warpAffine(image, mat, (64,64))
    return warped_image, target_image

random_transform_args = {
    'rotation_range': 20,
    'zoom_range': 0.1,
    'shift_range': 0.05,
    'random_flip': 0.5,
    }
def read_image(fn, random_transform_args=random_transform_args):
    image = cv2.imread(fn)
    image = cv2.resize(image, (256,256)) / 255 * 2 - 1
    image = random_transform( image, **random_transform_args)
    warped_img, target_img = random_warp_rev(image)    
    
    #if np.random.uniform() < 0.1: # random downscaling
    #    rand_size = np.random.randint(48) + 16
    #    warped_img = cv2.resize(warped_img, (rand_size,rand_size))
    #    warped_img = cv2.resize(warped_img, (64,64))
    #elif np.random.uniform() < 0.3:
    #    warped_img = random_cutout(warped_img)
    
    return warped_img, target_img

In [0]:
# A generator function that yields epoch, batchsize of warped_img and batchsize of target_img
def minibatch(data, batchsize):
    length = len(data)
    epoch = i = 0
    tmpsize = None  
    shuffle(data)
    while True:
        size = tmpsize if tmpsize else batchsize
        if i+size > length:
            shuffle(data)
            i = 0
            epoch+=1        
        rtn = np.float32([read_image(data[j]) for j in range(i,i+size)])
        i+=size
        tmpsize = yield epoch, rtn[:,0,:,:,:], rtn[:,1,:,:,:]       

def minibatchAB(dataA, batchsize):
    batchA = minibatch(dataA, batchsize)
    tmpsize = None    
    while True:        
        ep1, warped_img, target_img = batchA.send(tmpsize)
        tmpsize = yield ep1, warped_img, target_img

In [0]:
def showG(test_A, test_B, path_A, path_B):
    figure_A = np.stack([
        test_A,
        np.squeeze(np.array([path_A([test_A[i:i+1]]) for i in range(test_A.shape[0])])),
        np.squeeze(np.array([path_B([test_A[i:i+1]]) for i in range(test_A.shape[0])])),
        ], axis=1 )
    figure_B = np.stack([
        test_B,
        np.squeeze(np.array([path_B([test_B[i:i+1]]) for i in range(test_B.shape[0])])),
        np.squeeze(np.array([path_A([test_B[i:i+1]]) for i in range(test_B.shape[0])])),
        ], axis=1 )

    figure = np.concatenate([figure_A, figure_B], axis=0 )
    figure = figure.reshape((4,7) + figure.shape[1:])
    figure = stack_images(figure)
    figure = np.clip((figure + 1) * 255 / 2, 0, 255).astype('uint8')
    figure = cv2.cvtColor(figure, cv2.COLOR_BGR2RGB)

    display(Image.fromarray(figure))
    return figure
    
def showG_mask(test_A, test_B, path_A, path_B):
    figure_A = np.stack([
        test_A,
        (np.squeeze(np.array([path_A([test_A[i:i+1]]) for i in range(test_A.shape[0])])))*2-1,
        (np.squeeze(np.array([path_B([test_A[i:i+1]]) for i in range(test_A.shape[0])])))*2-1,
        ], axis=1 )
    figure_B = np.stack([
        test_B,
        (np.squeeze(np.array([path_B([test_B[i:i+1]]) for i in range(test_B.shape[0])])))*2-1,
        (np.squeeze(np.array([path_A([test_B[i:i+1]]) for i in range(test_B.shape[0])])))*2-1,
        ], axis=1 )

    figure = np.concatenate([figure_A, figure_B], axis=0 )
    figure = figure.reshape((4,7) + figure.shape[1:])
    figure = stack_images(figure)
    figure = np.clip((figure + 1) * 255 / 2, 0, 255).astype('uint8')
    figure = cv2.cvtColor(figure, cv2.COLOR_BGR2RGB)

    display(Image.fromarray(figure))
    return figure

<a id='10'></a>
# 10. Start Training

Show results and save model weights every `display_iters` iterations.

In [0]:
!mkdir models # create ./models directory

In [0]:
!mkdir previews

In [32]:
# Get filenames
train_A = load_data(img_dirA)
train_B = load_data(img_dirB)

assert len(train_A), "No image found in " + str(img_dirA)
assert len(train_B), "No image found in " + str(img_dirB)

print ("Number of images in folder A: " + str(len(train_A)))
print ("Number of images in folder B: " + str(len(train_B)))

Number of images in folder A: 538
Number of images in folder B: 661


In [0]:
t0 = time.time()
niter = 300
gen_iterations = 0
epoch = 0
errGA_sum = errGB_sum = errDA_sum = errDB_sum = errDA2_sum = errDB2_sum = errDA_code_sum = errDB_code_sum = 0

display_iters = 300
train_batchA = minibatchAB(train_A, batchSize)
train_batchB = minibatchAB(train_B, batchSize)

# ========== Change 10000 to desired iterations  ========== 
while gen_iterations < 50000: 
    epoch, warped_A, target_A = next(train_batchA) 
    epoch, warped_B, target_B = next(train_batchB) 
    
    # Train dicriminators for one batch
    if gen_iterations % 1 == 0:
        errDA = netDA_train([warped_A, target_A])
        errDB = netDB_train([warped_B, target_B])
        errDA2 = netDA2_train([warped_A, target_A])
        errDB2 = netDB2_train([warped_B, target_B])
        if gen_iterations >= 0:
            errDA_code = netDA_code_train([warped_A, target_A])
            errDB_code = netDB_code_train([warped_B, target_B])
        elif gen_iterations % 2 == 0:
            errDA_code = netDA_code_train([warped_A, target_A])
        else:
            errDB_code = netDB_code_train([warped_B, target_B])
        #errDA_feat = netDA_feat_train([warped_A, target_A])
        #errDB_feat = netDB_feat_train([warped_B, target_B])
    errDA_sum += errDA[0]
    errDB_sum += errDB[0]
    errDA2_sum += errDA2[0]
    errDB2_sum += errDB2[0]
    errDA_code_sum += errDA_code[0]
    errDB_code_sum += errDB_code[0]
    
    if gen_iterations == 5:
        print ("working.")
    
    #if gen_iterations == 3001:
    # Swap decoder at 3001th iter.
    #    decoder_A.load_weights("models/decoder_B.h5")
    #    decoder_B.load_weights("models/decoder_A.h5")

    
    # Train generators for one batch
    errGA = netGA_train([warped_A, target_A])
    errGB = netGB_train([warped_B, target_B])
    errGA_sum += errGA[0]
    errGB_sum += errGB[0]
    gen_iterations+=1
    
    if gen_iterations % display_iters == 0 or gen_iterations == 50:
        clear_output()
        print('[%d/%d][%d] Loss_DA: %f Loss_DB: %f Loss_GA: %f Loss_GB: %f time: %f'
        % (epoch, niter, gen_iterations, errDA_sum/display_iters, errDB_sum/display_iters,
           errGA_sum/display_iters, errGB_sum/display_iters, time.time()-t0))  
        print('Loss_DA_code: %f Loss_DB_code: %f Loss_DA2: %f Loss_DB2: %f' 
              % (errDA_code_sum/display_iters, errDB_code_sum/display_iters,
                 errDA2_sum/display_iters, errDB2_sum/display_iters))   
        
        # get new batch of images and generate results for visualization
        _, wA, tA = train_batchA.send(14)  
        _, wB, tB = train_batchB.send(14)
        preview_im = showG(tA, tB, path_A, path_B)   
        plt.imsave("./previews/"+str(gen_iterations)+"_masked.jpg", preview_im)
        print (" ")
        preview_im = showG(wA, wB, path_bgr_A, path_bgr_B)   
        plt.imsave("./previews/"+str(gen_iterations)+"_raw.jpg", preview_im)  
        print (" ")    
        preview_im = showG_mask(tA, tB, path_mask_A, path_mask_B)  
        plt.imsave("./previews/"+str(gen_iterations)+"_mask.jpg", preview_im) 
        if not gen_iterations == 50:
            errGA_sum = errGB_sum = errDA_sum = errDB_sum = 0
            errDA2_sum = errDB2_sum = 0
            errDA_code_sum = errDB_code_sum = 0
        
        # Save models
        encoder.save_weights("models/encoder.h5")
        decoder_A.save_weights("models/decoder_A.h5")
        decoder_B.save_weights("models/decoder_B.h5")
        netDA.save_weights("models/netDA.h5")
        netDB.save_weights("models/netDB.h5")
        netDA2.save_weights("models/netDA2.h5")
        netDB2.save_weights("models/netDB2.h5")
        #netDA_feat.save_weights("models/netDA_feat.h5")
        #netDB_feat.save_weights("models/netDB_feat.h5")
        netD_code.save_weights("models/netD_code.h5")

# 11. Video Conversion

In [0]:
from moviepy.editor import VideoFileClip

In [0]:
path_func = path_abgr_B

## MTCNN 

In [0]:
import mtcnn_detect_face

In [0]:
def create_mtcnn(sess, model_path):
    if not model_path:
        model_path,_ = os.path.split(os.path.realpath(__file__))

    with tf.variable_scope('pnet2'):
        data = tf.placeholder(tf.float32, (None,None,None,3), 'input')
        pnet = mtcnn_detect_face.PNet({'data':data})
        pnet.load(os.path.join(model_path, 'det1.npy'), sess)
    with tf.variable_scope('rnet2'):
        data = tf.placeholder(tf.float32, (None,24,24,3), 'input')
        rnet = mtcnn_detect_face.RNet({'data':data})
        rnet.load(os.path.join(model_path, 'det2.npy'), sess)
    with tf.variable_scope('onet2'):
        data = tf.placeholder(tf.float32, (None,48,48,3), 'input')
        onet = mtcnn_detect_face.ONet({'data':data})
        onet.load(os.path.join(model_path, 'det3.npy'), sess)
    return pnet, rnet, onet

In [35]:
WEIGHTS_PATH = "./mtcnn_weights/"

sess = K.get_session()
with sess.as_default():
    global pnet, rnet, onet 
    pnet2, rnet2, onet2 = create_mtcnn(sess, WEIGHTS_PATH)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [0]:
global pnet, rnet, onet
pnet_fun = K.function([pnet2.layers['data']],[pnet2.layers['conv4-2'], pnet2.layers['prob1']])
rnet_fun = K.function([rnet2.layers['data']],[rnet2.layers['conv5-2'], rnet2.layers['prob1']])
onet_fun = K.function([onet2.layers['data']], [onet2.layers['conv6-2'], onet2.layers['conv6-3'], onet2.layers['prob1']])

with tf.variable_scope('pnet2', reuse=True):
    data = tf.placeholder(tf.float32, (None,None,None,3), 'input')
    pnet2 = mtcnn_detect_face.PNet({'data':data})
    pnet2.load(os.path.join("./mtcnn_weights/", 'det1.npy'), sess)
with tf.variable_scope('rnet2', reuse=True):
    data = tf.placeholder(tf.float32, (None,24,24,3), 'input')
    rnet2 = mtcnn_detect_face.RNet({'data':data})
    rnet2.load(os.path.join("./mtcnn_weights/", 'det2.npy'), sess)
with tf.variable_scope('onet2', reuse=True):
    data = tf.placeholder(tf.float32, (None,48,48,3), 'input')
    onet2 = mtcnn_detect_face.ONet({'data':data})
    onet2.load(os.path.join("./mtcnn_weights/", 'det3.npy'), sess)

In [0]:
pnet = K.function([pnet2.layers['data']],[pnet2.layers['conv4-2'], pnet2.layers['prob1']])
rnet = K.function([rnet2.layers['data']],[rnet2.layers['conv5-2'], rnet2.layers['prob1']])
onet = K.function([onet2.layers['data']], [onet2.layers['conv6-2'], onet2.layers['conv6-3'], onet2.layers['prob1']])

## FCN8s for face segmentation

In [ ]:
from KCN8s_keras import FCN

In [0]:
fcn = FCN()

In [0]:
fcn.load_weights("Keras_FCN8s_face_seg_YuvalNirkin.h5")

In [0]:
def fcn_mask_seg(x):
    def vgg_preprocess(im):
        im = cv2.resize(im, (500, 500))
        in_ = np.array(im, dtype=np.float32)
        #in_ = in_[:,:,::-1]
        in_ -= np.array((104.00698793,116.66876762,122.67891434))
        in_ = in_[np.newaxis,:]
        #in_ = in_.transpose((2,0,1))
        return in_
    
    inp_fcn = vgg_preprocess(x)
    out_mask_fcn = fcn.predict([inp_fcn])
    out_mask_fcn = cv2.resize(np.squeeze(out_mask_fcn), (x.shape[1],x.shape[0]))
    out_mask_fcn = np.clip(out_mask_fcn.argmax(axis=2), 0, 1).astype(np.float64)
    return out_mask_fcn

## Video conversion functions

In [0]:
use_smoothed_mask = True
use_smoothed_bbox = True

def is_overlap(box1, box2):
    overlap_x0 = np.max([box1[0], box2[0]]).astype(np.float32)
    overlap_y1 = np.min([box1[1], box2[1]]).astype(np.float32)
    overlap_x1 = np.min([box1[2], box2[2]]).astype(np.float32)
    overlap_y0 = np.max([box1[3], box2[3]]).astype(np.float32)
    area_iou = (overlap_x1-overlap_x0) * (overlap_y1-overlap_y0)
    area_box1 = (box1[2]-box1[0]) * (box1[1]-box1[3])
    area_box2 = (box2[2]-box2[0]) * (box2[1]-box2[3])    
    return (area_iou / area_box1) >= 0.2
    
def remove_overlaps(faces):    
    main_face = get_most_conf_face(faces)
    main_face_bbox = main_face[0]
    result_faces = []
    result_faces.append(main_face_bbox)
    for (x0, y1, x1, y0, conf_score) in faces:
        if not is_overlap(main_face_bbox, (x0, y1, x1, y0)):
            result_faces.append((x0, y1, x1, y0, conf_score))
    return result_faces

def get_most_conf_face(faces):
    # Return the bbox w/ the highest confidence score
    best_conf_score = 0
    conf_face = None
    for (x0, y1, x1, y0, conf_score) in faces: 
        if conf_score >= best_conf_score:
            best_conf_score = conf_score
            conf_face = [(x0, y1, x1, y0, conf_score)]
    return conf_face

def kalmanfilter_init(noise_coef):
    kf = cv2.KalmanFilter(4,2)
    kf.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]], np.float32)
    kf.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]], np.float32)
    kf.processNoiseCov = noise_coef * np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], np.float32)
    return kf

def is_higher_than_480p(x):
    return (x.shape[0] * x.shape[1]) >= (858*480)

def is_higher_than_720p(x):
    return (x.shape[0] * x.shape[1]) >= (1280*720)

def is_higher_than_1080p(x):
    return (x.shape[0] * x.shape[1]) >= (1920*1080)

def calibrate_coord(faces, video_scaling_factor):
    for i, (x0, y1, x1, y0, _) in enumerate(faces):
        faces[i] = (x0*video_scaling_factor, y1*video_scaling_factor, 
                    x1*video_scaling_factor, y0*video_scaling_factor, _)
    return faces

def process_mtcnn_bbox(bboxes, im_shape):
    # outuut bbox coordinate of MTCNN is (y0, x0, y1, x1)
    # Process the bbox coord. to a square bbox with ordering (x0, y1, x1, y0)
    for i, bbox in enumerate(bboxes):
        y0, x0, y1, x1 = bboxes[i,0:4]
        w = int(y1 - y0)
        h = int(x1 - x0)
        length = (w + h) / 2
        center = (int((x1+x0)/2),int((y1+y0)/2))
        new_x0 = np.max([0, (center[0]-length//2)])#.astype(np.int32)
        new_x1 = np.min([im_shape[0], (center[0]+length//2)])#.astype(np.int32)
        new_y0 = np.max([0, (center[1]-length//2)])#.astype(np.int32)
        new_y1 = np.min([im_shape[1], (center[1]+length//2)])#.astype(np.int32)
        bboxes[i,0:4] = new_x0, new_y1, new_x1, new_y0
    return bboxes

def get_downscale_factor(image):
    if is_higher_than_1080p(image):
        return  4 + video_scaling_offset
    elif is_higher_than_720p(image):
        return 3 + video_scaling_offset
    elif is_higher_than_480p(image):
        return 2 + video_scaling_offset
    else:
        return 1

def get_faces_bbox(image):  
    global pnet, rnet, onet 
    global detec_threshold
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, detec_threshold ]  # three steps's threshold
    factor = 0.709 # scale factor
    if manually_downscale:
        video_scaling_factor = manual_downscale_factor
        resized_image = cv2.resize(image, 
                                   (image.shape[1]//video_scaling_factor, 
                                    image.shape[0]//video_scaling_factor))
        faces, pnts = mtcnn_detect_face.detect_face(resized_image, minsize, pnet, rnet, onet, threshold, factor)
        faces = process_mtcnn_bbox(faces, resized_image.shape)
        faces = calibrate_coord(faces, video_scaling_factor)
    else:
        video_scaling_factor = get_downscale_factor(image)
        resized_image = cv2.resize(image, 
                                   (image.shape[1]//video_scaling_factor, 
                                    image.shape[0]//video_scaling_factor))
        faces, pnts = mtcnn_detect_face.detect_face(image, minsize, pnet, rnet, onet, threshold, factor)
        faces = process_mtcnn_bbox(faces, image.shape)
        faces = calibrate_coord(faces, video_scaling_factor)
    return faces

def get_smoothed_coord(x0, x1, y0, y1, shape, ratio=0.65):
    global prev_x0, prev_x1, prev_y0, prev_y1
    if not use_kalman_filter:
        x0 = int(ratio * prev_x0 + (1-ratio) * x0)
        x1 = int(ratio * prev_x1 + (1-ratio) * x1)
        y1 = int(ratio * prev_y1 + (1-ratio) * y1)
        y0 = int(ratio * prev_y0 + (1-ratio) * y0)
    else:
        x0y0 = np.array([x0, y0]).astype(np.float32)
        x1y1 = np.array([x1, y1]).astype(np.float32)
        kf0.correct(x0y0)
        pred_x0y0 = kf0.predict()
        kf1.correct(x1y1)
        pred_x1y1 = kf1.predict()
        x0 = np.max([0, pred_x0y0[0][0]]).astype(np.int)
        x1 = np.min([shape[0], pred_x1y1[0][0]]).astype(np.int)
        y0 = np.max([0, pred_x0y0[1][0]]).astype(np.int)
        y1 = np.min([shape[1], pred_x1y1[1][0]]).astype(np.int)
        if x0 == x1 or y0 == y1:
            x0, y0, x1, y1 = prev_x0, prev_y0, prev_x1, prev_y1
    return x0, x1, y0, y1     
    
def set_global_coord(x0, x1, y0, y1):
    global prev_x0, prev_x1, prev_y0, prev_y1
    prev_x0 = x0
    prev_x1 = x1
    prev_y1 = y1
    prev_y0 = y0
    
def merge_mask(mask1, mask2,mode="avg"):
    mask2 = cv2.resize(mask2, (mask1.shape[1],mask1.shape[0]))
    if mode == "avg":
        return (mask1 + mask2).astype(np.float32) / 2
    elif mode == "max":
        return np.maximum(mask1, mask2)
    elif mode == "adaptive":
        sum1 = np.sum(mask1)/3
        sum2 = np.sum(mask2)
        ratio = np.clip(sum1/(sum1+sum2), 0.2, 0.8)
        return ratio*mask1 + (1-ratio)*mask2

def get_tta_masked_result(result_bgr, result_a, roi_image):
    result_bgr = cv2.resize(result_bgr, (64,64)).astype(np.float64)
    result_a = cv2.resize(result_a, (64,64)).astype(np.float64)
    roi_image_resized = cv2.resize(roi_image, (64,64)).astype(np.float64)
    result_a = cv2.GaussianBlur(result_a, (7,7), 6)[:,:,np.newaxis]
    result = (result_a/255 * result_bgr.astype(np.float64) + (1-result_a/255) * roi_image_resized).astype(np.uint8)
    result = cv2.resize(result, (roi_image.shape[1],roi_image.shape[0]))
    if use_color_correction:
        result = color_hist_match(result, roi_image).astype('uint8')
    result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB).astype('uint8')
    return result    

def generate_face(ae_input, path_abgr, roi_size, roi_image):
    result = np.squeeze(np.array([path_abgr([[ae_input]])]))
    result_a = result[:,:,0] * 255
    result_bgr = np.clip( (result[:,:,1:] + 1) * 255 / 2, 0, 255 )
    result_a_clear = np.copy(result_a)
    result_a = cv2.GaussianBlur(result_a ,(7,7),6)
    if use_landmark_match and False:
        resized_roi = cv2.resize(roi_image, (64,64))
        result_bgr, result_a = landmarks_match_mtcnn(resized_roi, result_bgr, result_a)
    #if use_color_correction:
    #    result_bgr = color_hist_match(result_bgr, roi_image)
    
    if use_FCN_mask:
        fcn_mask = fcn_mask_seg(roi_image)
        if merge_mask_mode == "avg":
            fcn_mask = merge_mask(fcn_mask, result[:,:,0], mode="avg")
        elif merge_mask_mode == "max":
            fcn_mask = merge_mask(fcn_mask, result[:,:,0], mode="max")
        elif merge_mask_mode == "adaptive":
            fcn_mask = merge_mask(fcn_mask, result[:,:,0], mode="adaptive")
        result_a = fcn_mask[:,:,np.newaxis] * 255
        result_a_clear = np.copy(fcn_mask * 255)
        fcn_mask_sz64 = cv2.resize(fcn_mask, (64,64))
        fcn_mask_sz64 = cv2.GaussianBlur(fcn_mask_sz64, (7,7), 6)[:,:,np.newaxis]
        result = (fcn_mask_sz64 * result_bgr.astype(np.float64) + (1-fcn_mask_sz64) * ((ae_input+1)*255/2)).astype(np.uint8)
    else:
        result_a = np.expand_dims(result_a, axis=2)
        result = (result_a/255 * result_bgr + (1 - result_a/255) * ((ae_input + 1) * 255 / 2)).astype('uint8')
        result = result_bgr.astype('uint8')
    if use_color_correction:
        result = color_hist_match(result, roi_image).astype('uint8')
    result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    result = cv2.resize(result, (roi_size[1],roi_size[0]))
    result_a_clear = np.expand_dims(cv2.resize(result_a_clear, (roi_size[1],roi_size[0])), axis=2)
    return result, result_a_clear, result_bgr

def get_init_mask_map(image):
    return np.zeros_like(image)

def get_init_comb_img(input_img):
    comb_img = np.zeros([input_img.shape[0], input_img.shape[1]*2,input_img.shape[2]])
    comb_img[:, :input_img.shape[1], :] = input_img
    comb_img[:, input_img.shape[1]:, :] = input_img
    return comb_img    

def get_init_triple_img(input_img, no_face=False):
    if no_face:
        triple_img = np.zeros([input_img.shape[0], input_img.shape[1]*3,input_img.shape[2]])
        triple_img[:, :input_img.shape[1], :] = input_img
        triple_img[:, input_img.shape[1]:input_img.shape[1]*2, :] = input_img      
        triple_img[:, input_img.shape[1]*2:, :] = (input_img * .15).astype('uint8')  
        return triple_img
    else:
        triple_img = np.zeros([input_img.shape[0], input_img.shape[1]*3,input_img.shape[2]])
        return triple_img

def get_mask(roi_image, h, w):
    mask = np.zeros_like(roi_image)
    mask[h//15:-h//15,w//15:-w//15,:] = 255
    mask = cv2.GaussianBlur(mask,(15,15),10)
    return mask

def hist_match(source, template):
    # Code borrow from:
    # https://stackoverflow.com/questions/32655686/histogram-matching-of-two-images-in-python-2-x
    oldshape = source.shape
    source = source.ravel()
    template = template.ravel()
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    t_values, t_counts = np.unique(template, return_counts=True)

    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)

def color_hist_match(src_im, tar_im):
    #src_im = cv2.cvtColor(src_im, cv2.COLOR_BGR2HSV)
    #tar_im = cv2.cvtColor(tar_im, cv2.COLOR_BGR2HSV)
    matched_0 = hist_match(src_im[:,:,0], tar_im[:,:,0])
    matched_1 = hist_match(src_im[:,:,1], tar_im[:,:,1])
    matched_2 = hist_match(src_im[:,:,2], tar_im[:,:,2])
    matched = np.stack((matched_1, matched_1, matched_2), axis=2).astype("uint8")
    #matched = cv2.cvtColor(matched, cv2.COLOR_HSV2BGR)
    return matched

def landmarks_match_mtcnn(source, target, alpha):
    global prev_pnts1, prev_pnts2
    ratio = 0.2
    """
    TODO: Reuse the landmarks of source image. Conceivable bug: coordinate mismatch.
    """
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.93 ]  # three steps's threshold
    factor = 0.709 # scale factor
    _, pnts1 = mtcnn_detect_face.detect_face(source, minsize, pnet, rnet, onet, threshold, factor) # redundant detection
    _, pnts2 = mtcnn_detect_face.detect_face(target, minsize, pnet, rnet, onet, threshold, factor)  
    
    if len(prev_pnts1) == 0 and len(prev_pnts2) == 0:
        if pnts1.shape[0] == 10 and pnts2.shape[0] == 10:
            prev_pnts1, prev_pnts2 = pnts1, pnts2        
    try:
        landmarks_XY1 = []
        landmarks_XY2 = []
        for i in range(5):
            landmarks_XY1.extend([((1-ratio)*pnts1[i+5][0] + ratio*prev_pnts1[i+5][0], 
                                   (1-ratio)*pnts1[i][0] + ratio*prev_pnts1[i][0])])
            landmarks_XY2.extend([((1-ratio)*pnts2[i+5][0] + ratio*prev_pnts2[i+5][0], 
                                   (1-ratio)*pnts2[i][0] + ratio*prev_pnts2[i][0])])
        M = umeyama(np.array(landmarks_XY1), np.array(landmarks_XY2), True)[0:2]
        result = cv2.warpAffine(source, M, (64, 64), borderMode=cv2.BORDER_REPLICATE)  
        mask = np.stack([alpha, alpha, alpha], axis=2)
        assert len(mask.shape) == 3, "len(mask.shape) is " + str(len(mask.shape))
        mask = cv2.warpAffine(mask, M, (64, 64), borderMode=cv2.BORDER_REPLICATE) 
        prev_landmarks_XY1, prev_landmarks_XY2 = landmarks_XY1, landmarks_XY2
        return result, mask[:,:,0].astype(np.float32)
    except:
        return source, alpha

def process_video(input_img): 
    global prev_x0, prev_x1, prev_y0, prev_y1
    global frames      
    global pnet, rnet, onet
    """The following if statement is meant to solve a bug that has an unknow cause."""
    if frames <= 2:
        with tf.variable_scope('pnet2', reuse=True):
            pnet2 = None
            data = tf.placeholder(tf.float32, (None,None,None,3), 'input')
            pnet2 = mtcnn_detect_face.PNet({'data':data})
            pnet2.load(os.path.join("./mtcnn_weights/", 'det1.npy'), sess)
        with tf.variable_scope('rnet2', reuse=True):
            rnet2 = None
            data = tf.placeholder(tf.float32, (None,24,24,3), 'input')
            rnet2 = mtcnn_detect_face.RNet({'data':data})
            rnet2.load(os.path.join("./mtcnn_weights/", 'det2.npy'), sess)
        with tf.variable_scope('onet2', reuse=True):
            onet2 = None
            data = tf.placeholder(tf.float32, (None,48,48,3), 'input')
            onet2 = mtcnn_detect_face.ONet({'data':data})
            onet2.load(os.path.join("./mtcnn_weights/", 'det3.npy'), sess)
        pnet = K.function([pnet2.layers['data']],
                          [pnet2.layers['conv4-2'], 
                           pnet2.layers['prob1']])
        rnet = K.function([rnet2.layers['data']],
                          [rnet2.layers['conv5-2'], 
                           rnet2.layers['prob1']])
        onet = K.function([onet2.layers['data']], 
                          [onet2.layers['conv6-2'], 
                           onet2.layers['conv6-3'], 
                           onet2.layers['prob1']])
    """Get face bboxes"""
    image = input_img
    faces = get_faces_bbox(image) # faces: face bbox coord
    
    """Init."""
    if len(faces) == 0:
        comb_img = get_init_comb_img(input_img)
        triple_img = get_init_triple_img(input_img, no_face=True)
    else:
        _ = remove_overlaps(faces) # Has non-max suppression already been implemented in MTCNN?        
    mask_map = get_init_mask_map(image)
    comb_img = get_init_comb_img(input_img)
    best_conf_score = 0
    
    """Process detected faces"""
    for (x0, y1, x1, y0, conf_score) in faces: 
        """Apply moving average bounding box"""        
        if use_smoothed_bbox:
            if frames != 0 and conf_score >= best_conf_score:
                x0, x1, y0, y1 = get_smoothed_coord(x0, x1, y0, y1, 
                                                    image.shape, 
                                                    ratio=0.65 if use_kalman_filter else bbox_moving_avg_coef)
                set_global_coord(x0, x1, y0, y1)
                best_conf_score = conf_score
                frames += 1
            elif conf_score <= best_conf_score:
                frames += 1
            else:
                if conf_score >= best_conf_score:
                    set_global_coord(x0, x1, y0, y1)
                    best_conf_score = conf_score
                if use_kalman_filter:
                    for i in range(200):
                        kf0.predict()
                        kf1.predict()
                frames += 1
        
        """ROI params"""
        h = int(x1 - x0)
        w = int(y1 - y0)
        roi_coef_h = 25
        roi_coef_w = 25
        roi_x0, roi_x1, roi_y0, roi_y1 = int(x0+h//roi_coef_h), int(x1-h//roi_coef_h), int(y0+w//roi_coef_w), int(y1-w//roi_coef_w)            
        cv2_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        roi_image = cv2_img[roi_x0:roi_x1,roi_y0:roi_y1,:]
        roi_size = roi_image.shape  
        
        """Feed face image into generator"""
        ae_input = cv2.resize(roi_image, (64,64))/255. * 2 - 1  
        result, result_a, result_bgr = generate_face(ae_input, path_func, roi_size, roi_image)
        
        """Apply test time augmentation"""
        if use_TTA:
            result_flip, result_a_flip, result_bgr_flip = generate_face(ae_input[:,::-1,:], 
                                                                        path_func, roi_size, 
                                                                        roi_image[:,::-1,:])
            result = (result.astype(np.float32) + result_flip[:,::-1,:].astype(np.float32))/2
            result_a = (result_a.astype(np.float32) + result_a_flip[:,::-1,:].astype(np.float32))/2
            result_bgr = (result_bgr.astype(np.float32) + result_bgr_flip[:,::-1,:].astype(np.float32))/2          
            result = result.astype('uint8')
            result_a = result_a.astype('uint8')
            result = get_tta_masked_result(result_bgr, result_a, roi_image)
            
        """Post processing"""
        if conf_score >= best_conf_score:
            mask_map[roi_x0:roi_x1,roi_y0:roi_y1,:] = result_a
            mask_map = np.clip(mask_map + .15 * input_img, 0, 255)     
        else:
            mask_map[roi_x0:roi_x1,roi_y0:roi_y1,:] += result_a
            mask_map = np.clip(mask_map, 0, 255)        
        if use_smoothed_mask:
            mask = get_mask(roi_image, h, w)
            roi_rgb = cv2.cvtColor(roi_image, cv2.COLOR_BGR2RGB)
            smoothed_result = mask/255 * result + (1-mask/255) * roi_rgb
            comb_img[roi_x0:roi_x1, input_img.shape[1]+roi_y0:input_img.shape[1]+roi_y1,:] = smoothed_result
        else:
            comb_img[roi_x0:roi_x1, input_img.shape[1]+roi_y0:input_img.shape[1]+roi_y1,:] = result
            
        triple_img = get_init_triple_img(input_img)
        triple_img[:, :input_img.shape[1]*2, :] = comb_img
        triple_img[:, input_img.shape[1]*2:, :] = mask_map
    
    """Return frame result"""
    global output_type
    if output_type == 1:
        return comb_img[:, input_img.shape[1]:, :]  # return only result image
    elif output_type == 2:
        return comb_img  # return input and result image combined as one
    elif output_type == 3:
        return triple_img #return input,result and mask heatmap image combined as one

## Video conversion config

In [0]:
use_kalman_filter = True
if use_kalman_filter:
    noise_coef = 8e-2 # Increase by 10x if tracking is slow. 
    kf0 = kalmanfilter_init(noise_coef)
    kf1 = kalmanfilter_init(noise_coef)
else:
    bbox_moving_avg_coef = 0.65
    
video_scaling_offset = 0 
manually_downscale = False
manual_downscale_factor = int(2) # should be an positive integer
use_color_correction = False
use_landmark_match = False # Under developement, This is not functioning.
use_TTA = False # test time augmentation
use_FCN_mask = False
merge_mask_mode = "adaptive" # avg, max, adaptive

# ========== Change the following line for different output type==========
# Output type: 
#    1. [ result ] 
#    2. [ source | result ] 
#    3. [ source | result | mask ]
global output_type
output_type = 3

# Detection threshold:  a float point between 0 and 1. Decrease this value if faces are missed.
global detec_threshold
detec_threshold = 0.7

## Start video conversion

In [ ]:
# Variables for smoothing bounding box
global prev_x0, prev_x1, prev_y0, prev_y1
global frames
global prev_pnts1, prev_pnts2
prev_x0 = prev_x1 = prev_y0 = prev_y1 = 0
frames = 0
prev_pnts1 = prev_pnts2 = np.array([])

output = 'OUTPUT_VIDEO.mp4'
clip1 = VideoFileClip("INPUT_VIDEO.mp4")
clip = clip1.fl_image(process_video).subclip(1, 10) #NOTE: this function expects color images!!
%time clip.write_videofile(output, audio=False)